In [ ]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

Let us create some data to play with.



---
y = mx + c

Let us add some noise:
𝕪 = y + N(0,σ)


In [ ]:
true_slope = 10
true_intercept = 20
true_sigma = 2

number_of_points = 200

x_values = np.linspace(0, 1, number_of_points)
true_y_values = true_slope * x_values + true_intercept
y_values = true_y_values + np.random.normal(scale=true_sigma, size=number_of_points)

true_parameters = {'slope': true_slope, 'intercept': true_intercept, 'sigma': true_sigma}

In [ ]:
plt.figure(figsize=(7,7))
p1 = plt.scatter(x_values, y_values)
p2, = plt.plot(x_values, true_y_values, color='r')
plt.xlabel('x', fontsize=20)
plt.ylabel('y', fontsize=20)
plt.legend((p1, p2), ('samples', 'true line'), fontsize=18)

Let us fit a non-bayesian linear model

In [ ]:
regressor_function = LinearRegression()
regressor_function.fit(x_values.reshape(-1,1), y_values)
predictions = regressor_function.predict(x_values.reshape(-1,1))
residuals = predictions - y_values

print('True Model:')
print('y_true = %s*x + %s'%(true_slope, true_intercept))
print('True sigma: %s\n'%true_parameters['sigma'])

print('Estimated Model:')
print('y_hat = %s*x + %s'%(regressor_function.coef_[0], regressor_function.intercept_))
print('Sd Residuals: %s'%(residuals.std()))

maximum_likelihood_estimates = {'slope': regressor_function.coef_[0], 'intercept': regressor_function.intercept_, 'sigma': residuals.std()} 

Fitting bayesian model

In [ ]:
maximum_likelihood_estimates

Setting Priors

m ~ N(0,100), 
c ~ N(0,100), 
σ ~ Exp(1)

Likelihood

y|m,c,σ ~ N(mx+c, σ)

Posterior

m,c,σ|y ~ ?

p(m,c,σ|y) ∝ p(y|m,c,σ) x p(m) x p(c) x p(σ)

In [ ]:
with pm.Model() as bayesian_regression_linear_model:
    #priors
    sigma = pm.Exponential("sigma", lam=1.0)
    intercept = pm.Normal("intercept", mu=0, sigma=20)
    slope = pm.Normal("slope", mu=0, sigma=20)

    #Likelihood
    likelihood = pm.Normal("y", mu=slope*x_values + intercept, sigma=sigma, observed=y_values)

    #posterior
    trace = pm.sample(10000, cores=4)

In [ ]:
plt.figure(figsize=(7, 7))
pm.traceplot(trace)
plt.tight_layout()

In [ ]:
for parameter in ['slope', 'sigma', 'intercept']:
    plt.figure(figsize=(10,4))
    trace_values = trace.get_values(parameter)
    mean, lower, upper = round(trace_values.mean(),2), round(trace_values.mean()-trace_values.std(),2), round(trace_values.mean()+trace_values.std(),2)
    sns.distplot(trace_values)
    posterior_estimate = plt.axvline(mean, color='b')
    maximum_likelihood_estimate = plt.axvline(maximum_likelihood_estimates[parameter], color='b',linestyle="dotted")
    plt.axvline(lower, color='r', linestyle='--')
    plt.axvline(upper, color='r', linestyle='--')
    plt.title('%s [True = %s]\nPosterior Mean: %s\nBound: (%s, %s)'%(parameter,true_parameters[parameter],mean,lower,upper), fontsize=20)
    true_values = plt.axvline(true_parameters[parameter], color='k')
    
    plt.legend((true_values, maximum_likelihood_estimate, posterior_estimate), ('true', 'MLE', 'Posterior Mean'), fontsize=18)
     
    plt.show()